In [0]:
!pip install aguaclara

In [0]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np

Below is an example method for pumping water and a coagulant. The coagulant stock is designed to ensure that the time between coagulant pulses is short enough that mixing will occur in the flocculator. 

In [0]:
D = 1 * u.inch
A = ac.area_circle(D)
v_up = 1 * u.mm/u.s
Q_water = (A * v_up).to(u.mL/u.s)
print('The water flow rate is',Q_water)

# Use an easy-load pump head with tubing codes 14, 16, 17, or 18. 
# Tubing 14 won't work because the 50 rpm pump can't deliver the required flow
# The choice of which tubing to use is somewhat arbitrary. High rpm wears out the tubing faster
#. Low rpm likely has bigger pulsations. Your choice! Here I compromise...
Water_tubingcode = 17
# find the volume per rev for Masterflex L/S pump heads
Pump_water_per_rev = ac.vol_per_rev_LS(id_number = Water_tubingcode)
print('The volume of water per revolution is',Pump_water_per_rev)
Pump_water_rpm = (Q_water/Pump_water_per_rev).to(u.rpm)
print('The water pump rpm is',Pump_water_rpm, 'for tubing size', Water_tubingcode)

# The coagulant dose range is 0.5 to 8 mg/L of aluminum.
# We need the peristaltic pump pulses to be very fast compared with the residence time of the flocculator
# Let's assume flocculator residence time 5 minutes (300 seconds)
# Set max pulse time to be 10% of flocculator residence time (3 seconds)
# Ismatec pump heads have 6 rollers, thus 6 pulses per rev
# minimum time for a pump revolution is 18 s
Pump_dosing_min_rpm = (1/(5*u.min * 0.1 * 6/u.turn)).to(u.rpm)
print('The minimum dosing pump rpm is',Pump_dosing_min_rpm)
# The coagulant stock has 70 grams/L of aluminum
C_PACl_super_stock = 70 * u.g/u.L
# Define a set of coagulant dosages over the target range with a logarithmic scaling
# the number of coagulant dosages to test. This doesn't include an extra "free" starting value of 0 to create a control that ProCoDA adds automatically
Max_x = 5
dose_min = 0.5 * u.mg/u.L
dose_max = 8 * u.mg/u.L
C_PACl_dose = np.logspace(np.log10(1),np.log10(dose_max/dose_min),Max_x)* dose_min
print('The coagulant dosages in the geometric series are',C_PACl_dose.magnitude, C_PACl_dose.units)
# Find the multiplication factor (the base) between each dose
base = (dose_max/dose_min)**(1/(Max_x-1))
print('The base of the geometric series is',base)
# The coefficient for our increment function is the minimum dose.

# Use the smallest tubing possible so we can use as high a concentration of PACl as possible.
# The smallest diameter tubing that we like to work with is 1.52 mm yellow-blue.
Pump_coag_per_rev = ac.vol_per_rev_3_stop('yellow-blue')
print('The coagulant tubing volume per turn is',Pump_coag_per_rev)
Coag_min_Q = (Pump_coag_per_rev*Pump_dosing_min_rpm).to(u.mL/u.s)
print('The minimum coagulant flow is',Coag_min_Q)

# now we can find the maximum coagulant stock concentration using the min dose and min flow rate
C_PACl_stock = ac.floor_step(dose_min*Q_water/Coag_min_Q,10*u.mg/u.L)

print('The PACl stock concentration is',C_PACl_stock)

# Now find the flow of the PACl stock for these target dosages
Q_PACl_stock = Q_water*C_PACl_dose/C_PACl_stock

print('The coagulant flow rate is',Q_PACl_stock.magnitude, 'mL/s')

Pump_coag_rpm = (Q_PACl_stock/Pump_coag_per_rev).to(u.rpm)
print('The pump rpm for the different flow rates are',Pump_coag_rpm.magnitude, Pump_coag_rpm.units)

The water flow rate is 0.5067 milliliter / second
The volume of water per revolution is 2.8 milliliter / turn
The water pump rpm is 10.86 revolutions_per_minute for tubing size 17
The minimum dosing pump rpm is 0.3333 revolutions_per_minute
The coagulant dosages in the geometric series are [0.5 1.  2.  4.  8. ] milligram / liter
The base of the geometric series is 2 dimensionless
The coagulant tubing volume per turn is 0.1488 milliliter / rev
The minimum coagulant flow is 0.0008269 milliliter / second
The PACl stock concentration is 300 milligram / liter
The coagulant flow rate is [0.00084451 0.00168902 0.00337805 0.0067561  0.0135122 ] mL/s
The pump rpm for the different flow rates are [0.34042406 0.68084811 1.36169622 2.72339245 5.4467849 ] revolutions_per_minute


# Clay Stock

Max kaolin clay stock concentration that we have tested is 150 g/L. Thus we know that it is possible to stir that suspension and feed it with a peristaltic pump. That was for a turbulent flow flocculator and a much higher flow rate than we commonly use in the AguaClara research lab. The maximum clay stock concentration is set by the resolution of the pump rpm which is 0.1 rpm for the Golander pumps.

100 mg/L of Humic acid was sufficient to prevent the clay stock from showing signs of flocculation. Presumably this ratio of 100 mg of humic acid per 15 g of clay can be used for other clay stock suspensions.

In [0]:
Clay_dose = 100 * u.NTU
Clay_stock_V = 1 * u.L
Pump_clay_per_rev = ac.vol_per_rev_3_stop('yellow-blue')

print('The clay tubing volume per turn is',Pump_clay_per_rev)
#set min pump speed to give a 2.5% adjustment
Pump_clay_min_rpm = 0.1/0.025 * u.rev/u.min
print('The minimum clay pump rpm is',Pump_clay_min_rpm)
Clay_min_Q = (Pump_clay_per_rev*Pump_clay_min_rpm).to(u.mL/u.s)
print('The minimum clay flow is',Clay_min_Q)

# now we can find the maximum coagulant stock concentration using the min dose and min flow rate
C_clay_stock = ac.floor_step(Clay_dose*Q_water/Clay_min_Q,1*u.mg/u.L)

print('The clay stock concentration is',C_clay_stock.to(u.g/u.L))

Clay_stock_t = (Clay_stock_V/Clay_min_Q).to(u.day)
print('The stock will last',Clay_stock_t)

The clay tubing volume per turn is 0.1488 milliliter / rev
The minimum clay pump rpm is 4 rev / minute
The minimum clay flow is 0.009923 milliliter / second
The clay stock concentration is 7.506 gram / liter
The stock will last 1.166 day


See [CEE 4530 lab manual on pumps](https://monroews.github.io/EnvEngLabTextbook/ProCoDA/ProCoDA.html#id15) for the mL/rev for various types of pump tubing.
